*MOOC Completion Prediction*
1. Data Preprocessing

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

In [12]:
# Importing the dataset
df = pd.read_csv("MOOC_AGG_1NF_all.csv")
df.head()


,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed
0,1,C_course-v1:TsinghuaX+00740043X_2015_T2+sp,0,7697,8439.0,34,18,0.156522,0.912075
1,1,C_course-v1:TsinghuaX+00740113_1X+sp,0,1618,1612.0,10,3,0.083333,1.003722
2,1,C_course-v1:TsinghuaX+30240184+sp,0,1386,1155.0,7,5,0.021277,1.200000
3,1,C_course-v1:TsinghuaX+30240243X+sp,0,2614,2612.0,7,4,0.029197,1.000766
4,1,C_course-v1:TsinghuaX+00740113_2X+sp,1,527,1684.0,5,2,0.036364,0.312945


In [13]:
# read another dataset
df2 = pd.read_csv("MOOC_Course_count.csv")
df2['counts'] = df2['counts'].astype(int)

# print type for each column
print(df2.dtypes)
print(df.dtypes)


course_id    object
counts        int64
dtype: object
user_id                  int64
course_id               object
dropped                  int64
local_watch              int64
video_total            float64
watching_count           int64
episodes_watched         int64
episodes_percentile    float64
watching_speed         float64
dtype: object


In [14]:
df2.head()

,course_id,counts
0,C_course-v1:MITx+6_002x+sp,510
1,C_course-v1:XJTU+C00204+sp,83
2,C_course-v1:XJTU+C00204+2019_T1,83
3,C_course-v1:MITx+2_03x+sp,81
4,C_course-v1:XJTU+PHLS100914+sp,85


In [15]:
# join tables to get information
df_new = pd.merge(df, df2, how="left", on="course_id")
df_new.head()

,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,counts
0,1,C_course-v1:TsinghuaX+00740043X_2015_T2+sp,0,7697,8439.0,34,18,0.156522,0.912075,115
1,1,C_course-v1:TsinghuaX+00740113_1X+sp,0,1618,1612.0,10,3,0.083333,1.003722,36
2,1,C_course-v1:TsinghuaX+30240184+sp,0,1386,1155.0,7,5,0.021277,1.200000,235
3,1,C_course-v1:TsinghuaX+30240243X+sp,0,2614,2612.0,7,4,0.029197,1.000766,137
4,1,C_course-v1:TsinghuaX+00740113_2X+sp,1,527,1684.0,5,2,0.036364,0.312945,55


In [16]:
# feature engineering

df_new.rename(columns={'video_count': 'episodes_watched', 'video_duration': 'video_total', "completion" : "dropped"}, inplace=True)
df_new['episodes_percentile'] = df_new.apply(lambda x: x['episodes_watched'] / x['counts'], axis=1)
df_new['watching_speed'] = df_new.apply(lambda x: x['local_watch'] / x['video_total'], axis=1)
df_new.drop(['counts'], axis=1, inplace=True)

df_new.head()




,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed
0,1,C_course-v1:TsinghuaX+00740043X_2015_T2+sp,0,7697,8439.0,34,18,0.156522,0.912075
1,1,C_course-v1:TsinghuaX+00740113_1X+sp,0,1618,1612.0,10,3,0.083333,1.003722
2,1,C_course-v1:TsinghuaX+30240184+sp,0,1386,1155.0,7,5,0.021277,1.200000
3,1,C_course-v1:TsinghuaX+30240243X+sp,0,2614,2612.0,7,4,0.029197,1.000766
4,1,C_course-v1:TsinghuaX+00740113_2X+sp,1,527,1684.0,5,2,0.036364,0.312945


In [17]:
dfSchoolCourse = pd.read_csv("schoolCourse.csv")

In [18]:
dfSchoolCourse.head()

,school_id,course_id
0,S_BNU,C_course-v1:BNU+CSL21148501+2018_T2
1,S_BNU,C_course-v1:BNU+GE310141091+2019_T1
2,S_BNU,C_course-v1:BNU+2017112001X+2019_T1
3,S_BNU,C_course-v1:BNU+2018122603X+2018_T2
4,S_BNU,C_course-v1:BNU+CSL21126882+2019_T1


In [19]:
df_new = pd.merge(df_new, dfSchoolCourse, how="left", on="course_id")
df_new.head()

,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,school_id
0,1,C_course-v1:TsinghuaX+00740043X_2015_T2+sp,0,7697,8439.0,34,18,0.156522,0.912075,S_TsinghuaX
1,1,C_course-v1:TsinghuaX+00740113_1X+sp,0,1618,1612.0,10,3,0.083333,1.003722,S_TsinghuaX
2,1,C_course-v1:TsinghuaX+30240184+sp,0,1386,1155.0,7,5,0.021277,1.200000,S_TsinghuaX
3,1,C_course-v1:TsinghuaX+30240243X+sp,0,2614,2612.0,7,4,0.029197,1.000766,S_TsinghuaX
4,1,C_course-v1:TsinghuaX+00740113_2X+sp,1,527,1684.0,5,2,0.036364,0.312945,S_TsinghuaX


2. Create dictionary to convert string to int and enable the model to process the data

In [20]:
# create a dictionary for string to int conversion
dict_1 = {}
for i, k in enumerate(df_new['course_id'].unique()):
    dict_1[k] = i
    
dict_2 = {}
for l, m in enumerate(df_new['school_id'].unique()):
    dict_2[m] = l

# apply that dictionary to the dataset

for i in range(len(df_new['course_id'])):
    df_new['course_id'][i] = dict_1.get(df_new['course_id'][i])
    
for l in range(len(df_new['school_id'])):
    df_new['school_id'][l] = dict_2.get(df_new['school_id'][l])



df_new.head()


/var/folders/yq/2m5984j90xsccjq9x78pp9ww0000gn/T/ipykernel_12862/40087727.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['course_id'][i] = dict_1.get(df_new['course_id'][i])
/var/folders/yq/2m5984j90xsccjq9x78pp9ww0000gn/T/ipykernel_12862/40087727.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['school_id'][l] = dict_2.get(df_new['school_id'][l])


,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,school_id
0,1,0,0,7697,8439.0,34,18,0.156522,0.912075,0
1,1,1,0,1618,1612.0,10,3,0.083333,1.003722,0
2,1,2,0,1386,1155.0,7,5,0.021277,1.200000,0
3,1,3,0,2614,2612.0,7,4,0.029197,1.000766,0
4,1,4,1,527,1684.0,5,2,0.036364,0.312945,0


In [21]:
df_new.tail()

,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,school_id
227929,41344,188,0,361,641.0,5,2,0.017241,0.563183,31
227930,41344,134,0,1528,1359.0,5,4,0.042553,1.124356,31
227931,41344,112,0,540,538.0,3,3,0.030928,1.003717,0
227932,41344,0,1,73,591.0,1,1,0.008696,0.123519,0
227933,41344,12,1,34,330.0,1,1,0.006024,0.103030,0


In [22]:
df_new.head(20)

,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,school_id
0,1,0,0,7697,8439.0,34,18,0.156522,0.912075,0
1,1,1,0,1618,1612.0,10,3,0.083333,1.003722,0
2,1,2,0,1386,1155.0,7,5,0.021277,1.200000,0
3,1,3,0,2614,2612.0,7,4,0.029197,1.000766,0
4,1,4,1,527,1684.0,5,2,0.036364,0.312945,0
5,2,5,0,239,680.0,6,4,0.078431,0.351471,1
6,3,6,0,1951,1733.0,20,3,0.081081,1.125793,2
7,3,7,0,1019,1018.0,6,1,0.027778,1.000982,3
8,4,8,0,25265,18571.0,60,28,0.368421,1.360454,0
9,4,9,0,985,1442.0,5,3,0.073171,0.683079,0


In [23]:
training_data, testing_data = train_test_split(df_new, test_size=0.2, random_state=25)
testing_data.head()

,user_id,course_id,dropped,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed,school_id
1699,318,23,0,569,1946.0,13,4,0.049383,0.292395,0
104452,18989,87,0,9012,7145.0,71,14,0.125000,1.261302,25
15058,2755,111,1,8295,8679.0,63,14,0.093333,0.955755,27
221723,40241,62,0,36345,4706.0,61,8,0.095238,7.723119,0
11168,2040,174,0,1668,1268.0,27,16,0.216216,1.315457,44


In [24]:
actaul_training_data1 = training_data.drop(['dropped'], axis=1)
actaul_training_data = actaul_training_data1.drop(['user_id'], axis=1)
actaul_training_data.head()

actaul_training_label = training_data['dropped']

actaul_training_data.describe()


,local_watch,video_total,watching_count,episodes_watched,episodes_percentile,watching_speed
count,1.823470e+05,182347.000000,182347.000000,182347.000000,182347.000000,182347.000000
mean,1.001487e+04,7534.285286,81.357171,14.465025,0.156127,1.042955
std,2.593600e+04,10809.203539,209.506931,23.485315,0.196244,3.006726
min,0.000000e+00,15.000000,1.000000,1.000000,0.001961,0.000000
25%,4.800000e+02,1159.000000,6.000000,2.000000,0.026786,0.423180
50%,2.074000e+03,3022.000000,19.000000,5.000000,0.071429,0.740958
75%,8.393500e+03,9002.000000,70.000000,16.000000,0.195122,1.032295
max,1.659818e+06,156099.000000,11832.000000,271.000000,1.600000,426.111111


3. Testing different models (random forest, adaboost, gradientboost)

In [25]:

#Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=445, max_depth=10)
rf.fit(actaul_training_data, actaul_training_label)



RandomForestClassifier(max_depth=10, random_state=445)

In [22]:

# cross validation #same values as andrew's
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, actaul_training_data, actaul_training_label)
print("Cross-validated scores:", scores)
print("Mean:", scores.mean())



Cross-validated scores: [0.79679188 0.79149986 0.79297486 0.79541528 0.7931668 ]
Mean: 0.7939697370821978


In [26]:
actaul_testing_data1 = testing_data.drop(['dropped'], axis=1)
actaul_testing_data = actaul_testing_data1.drop(['user_id'], axis=1)
actaul_testing_data.head()

actaul_testing_label = testing_data['dropped']


In [27]:
# Predicting the Test set results
#better than andrew's
y_pred = rf.predict(actaul_testing_data)

#testing scores
print("Accuracy:",sklearn.metrics.accuracy_score(actaul_testing_label, y_pred))
print("Precision:",sklearn.metrics.precision_score(actaul_testing_label, y_pred))
print("Recall:",sklearn.metrics.recall_score(actaul_testing_label, y_pred))
print("F1:",sklearn.metrics.f1_score(actaul_testing_label, y_pred))



Accuracy: 0.8008642814837563
Precision: 0.6869798409469207
Recall: 0.5661484529797287
F1: 0.6207386363636362


In [31]:
# test AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=10)
ada = AdaBoostClassifier(base_estimator=dt, n_estimators=100, random_state=445)
ada.fit(actaul_training_data, actaul_training_label)

#testing scores
y_pred_adp = ada.predict(actaul_testing_data)
print("Accuracy:",sklearn.metrics.accuracy_score(actaul_testing_label, y_pred_adp))
print("Precision:",sklearn.metrics.precision_score(actaul_testing_label, y_pred_adp))
print("Recall:",sklearn.metrics.recall_score(actaul_testing_label, y_pred_adp))
print("F1:",sklearn.metrics.f1_score(actaul_testing_label, y_pred_adp))



Accuracy: 0.7871761686445697
Precision: 0.6355636594260345
Recall: 0.6109586953208352
F1: 0.6230183400683866


In [32]:
# test GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
"""
#grid search
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [100, 200, 300, 400, 500], 'max_depth': [5, 10, 15, 20, 25]}
gb = GradientBoostingClassifier(random_state=445)
grid_search = GridSearchCV(gb, param_grid, cv=5, n_jobs=-1)

grid_search.fit(actaul_training_data, actaul_training_label)
print(grid_search.best_params_)
"""

"\n#grid search\nfrom sklearn.model_selection import GridSearchCV\nparam_grid = {'n_estimators': [100, 200, 300, 400, 500], 'max_depth': [5, 10, 15, 20, 25]}\ngb = GradientBoostingClassifier(random_state=445)\ngrid_search = GridSearchCV(gb, param_grid, cv=5, n_jobs=-1)\n\ngrid_search.fit(actaul_training_data, actaul_training_label)\nprint(grid_search.best_params_)\n"

In [33]:
gb = GradientBoostingClassifier(random_state=445, n_estimators=300, max_depth=10)
gb.fit(actaul_training_data, actaul_training_label)

#testing scores
y_pred_gb = gb.predict(actaul_testing_data)
print("Accuracy:",sklearn.metrics.accuracy_score(actaul_testing_label, y_pred_gb))
print("Precision:",sklearn.metrics.precision_score(actaul_testing_label, y_pred_gb))
print("Recall:",sklearn.metrics.recall_score(actaul_testing_label, y_pred_gb))
print("F1:",sklearn.metrics.f1_score(actaul_testing_label, y_pred_gb))



Accuracy: 0.8297102244060807
Precision: 0.7226422933111757
Recall: 0.662780064014632
F1: 0.6914178956155345


In [37]:
#attempt XGBoost
from xgboost import XGBClassifier

actaul_training_data['course_id'] = actaul_training_data['course_id'].astype('int')

actaul_training_data.dtypes

actaul_testing_data['course_id'] = actaul_testing_data['course_id'].astype('int')

actaul_training_data['school_id'] = actaul_training_data['school_id'].astype('int')

actaul_training_data.dtypes

actaul_testing_data['school_id'] = actaul_testing_data['school_id'].astype('int')



In [38]:
xgb = XGBClassifier(random_state=445, n_estimators=300, max_depth=10)
xgb.fit(actaul_training_data, actaul_training_label)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=445, ...)

In [39]:
# testing scores
y_pred_xgb = xgb.predict(actaul_testing_data)
print("Accuracy:",sklearn.metrics.accuracy_score(actaul_testing_label, y_pred_xgb))
print("Precision:",sklearn.metrics.precision_score(actaul_testing_label, y_pred_xgb))
print("Recall:",sklearn.metrics.recall_score(actaul_testing_label, y_pred_xgb))
print("F1:",sklearn.metrics.f1_score(actaul_testing_label, y_pred_xgb))


Accuracy: 0.8257836663961217
Precision: 0.7150448356027898
Recall: 0.6563023929279074
F1: 0.6844154812048002


4. Unsupservised Learning

In [63]:
# K-Means Clustering


ValueError: Experimental support for categorical data is not implemented for current tree method yet.